# APDS-9960 Color Data Playground

Processing the APDS-9960 color data is interesting, since it provides both RGB data and clear data. 

That's cool and all, but a lot of use cases call for two specific types of data that can also be useful.

* Color Temperature (CT)
* Lux (ambient light level)

## Color Engine Configuration

Adding to this is the complexity of sensing color data, due a few key tuning options for the color engine.

* Integration Time
    * `ATIME` (8-bit value with 255 options)
* Gain
    * `CONTROL<AGAIN>` (2-bit value with 4 options)

## Color Engine Interrupts

In addition, the sensor's interrupt pin can be used to alert the host device to two different types of interrupt conditions. Both are triggered by results from the clear channel, ignoring the R/G/B values.

The sensor's interrupt pin can also be assrted when `AINT` or `ASAT` are triggered.

* Clear channel values below over a threshold window
    * `AINT`, controlled by low (`AILTL`/`ALHTL`) and high (`AILTH`/`AIHTH`) thresholds.
    * Asserts int pin if `ENABLE<AIEN>` is true
* Clear channel analog circuit saturation 
    * `ASAT`
    * Asserts int pin if `CONFIG2<CPSIEN>` is true

The `AINT` interrupt can be useful for handling light-change conditions, like detecting lights turning on or off, detecting sunrise/sunset, or 

The `ASAT` interrupt can be useful for, for instance, responsively tuning the sensor's `AGAIN` to respond to saturation conditions caused by exceptionally bright lighting conditions.

These can be cleared via `CICLEAR` and are also cleared by the broader `AICLEAR`. 

## Color Data Structure

The data available from the sensor is provided in four 16-bit values from its ADC, which reads in four photodiodes masked with R, G, B and "clear" masks. This is quite different from the proximity and gesture engines, which only offer 8-bit values.

These become available in four two-byte (high/low) registers. When new data is available, `AVALID` flips to true. `AVALID` becomes reset whenever any of the data registers are read.

## Processing Color Data

Getting CT/Lux from the RGB/C data is a bit involved though. The "clear" channel could be useful for lux but its results are heavily variable based on integration time and gain settings. Similiarly, the R/G/B values can vary widely based on integration time and gain.

Some useful references:

* <https://andi-siess.de/rgb-to-color-temperature/>
* <http://www.vendian.org/mncharity/dir3/blackbody/UnstableURLs/bbr_color.html>
* <https://github.com/adafruit/Adafruit_CircuitPython_TCS34725/blob/main/adafruit_tcs34725.py>
* <https://github.com/adafruit/Adafruit_TCS34725/blob/master/Adafruit_TCS34725.cpp>
* <https://www.usna.edu/Users/oceano/raylee/papers/RLee_AO_CCTpaper.pdf>
    * "McCamy's Forumla" for CCT calculation

### Color Temperature Processing

However, the ratio between the R/G/B values can give us the data we need for this.

There are a few different ways to go about calculating CCT from RGB. Arduino libraries for this sensor and others like it tend to reuse the same basic code which was well-Pythonized in the original APDS-9960 driver within `colorutil`.

Here's what that looks like in `colorutil`.

In [ ]:
def calculate_color_temperature(r: int, g: int, b: int) -> float:
    """Converts the raw R/G/B values to color temperature in degrees Kelvin"""

    #  1. Map RGB values to their XYZ counterparts.
    #   Based on 6500K fluorescent, 3000K fluorescent
    #    and 60W incandescent values for a wide range.
    #    Note: Y = Illuminance or lux
    x = (-0.14282 * r) + (1.54924 * g) + (-0.95641 * b)
    y = (-0.32466 * r) + (1.57837 * g) + (-0.73191 * b)
    z = (-0.68202 * r) + (0.77073 * g) + (0.56332 * b)

    #  Avoid divide-by-zero errors
    if (x + y + x) != 0:
        #  2. Calculate the chromaticity co-ordinates
        xchrome = x / (x + y + z)
        ychrome = y / (x + y + z)

        #  3. Use   to determine the CCT
        n = (xchrome - 0.3320) / (0.1858 - ychrome)
    else:
        xchrome = 0
        ychrome = 0
        n = 0

    #  4. Calculate the final CCT
    cct = (449.0 * pow(n, 3)) + (3525.0 * pow(n, 2)) + (6823.3 * n) + 5520.33

    #    Return the results in degrees Kelvin
    return cct

def calculate_lux(r: int, g: int, b: int) -> float:
    """Calculate ambient light values"""
    #   This only uses RGB ... how can we integrate clear or calculate lux
    #   based exclusively on clear since this might be more reliable?
    illuminance = (-0.32466 * r) + (1.57837 * g) + (-0.73191 * b)

    return illuminance

There's a similar algorithm in the `TCS34725` Ardunio driver, which looks basically identical to `colorutil`. A very similar algorithm is implemented quite a bit more "pythonically" in the CPy driver too, but this looks like its closely tied to the specific sensor based on values provided by the vendor, Taos/AMS, in an application note, `DN40`

* <https://ams.com/documents/20143/36005/ColorSensors_AN000166_1-00.pdf/d1290c78-4ef1-5b88-bff0-8e80c2f92b6b>

S

